# Visualizing Results in Weights and Biases

With the Weights and Biases integration, you can now spend more time extracting deeper insights into your evaluation results. The integration is designed to streamline the process of logging and visualizing experiment results using the Weights & Biases (W&B) platform.

The integration provide functionalities

- to automatically log the evaluation results,
- log the samples as W&B Tables for easy visualization,
- log the `results.json` file as an artifact for version control,
- log the `<task_name>_eval_samples.json` file if the samples are logged,
- generate a comprehensive report for analysis and visualization with all the important metric,
- log task and cli configs,
- and more out of the box like the command used to run the evaluation, GPU/CPU counts, timestamp, etc.

The integration is super easy to use with the eval harness. Let's see how!

In [ ]:
# Install this project if you did not already have it.
# This is all that is needed to be installed to start using Weights and Biases

!pip -qq install -e ..[wandb]

# Run the Eval Harness

Run the eval harness as usual with a `wandb_args` flag. This flag is used to provide arguments for initializing a wandb run ([wandb.init](https://docs.wandb.ai/ref/python/init)) as comma separated string arguments.

If `wandb_args` flag is used, the metrics and all other goodness will be automatically logged to Weights and Biases. In the stdout, you will find the link to the W&B run page as well as link to the generated report.

## Set your API Key

Before you can use W&B, you need to authenticate your machine with an authentication key. Visit https://wandb.ai/authorize to get one.

In [ ]:
import wandb


wandb.login()

> Note that if you are using command line you can simply authenticate your machine by doing `wandb login` in your terminal. For more info check out the [documentation](https://docs.wandb.ai/quickstart#2-log-in-to-wb).

## Run and log to W&B

In [ ]:
!lm_eval \
    --model hf \
    --model_args pretrained=microsoft/phi-2,trust_remote_code=True \
    --tasks hellaswag,mmlu_abstract_algebra \
    --device cuda:0 \
    --batch_size 8 \
    --output_path output/phi-2 \
    --limit 10 \
    --wandb_args project=lm-eval-harness-integration \
    --log_samples

W&B can also be initialized programmatically for use outside the CLI to parse and log the results.

In [ ]:
import lm_eval
from lm_eval.loggers import WandbLogger


results = lm_eval.simple_evaluate(
    model="hf",
    model_args="pretrained=microsoft/phi-2,trust_remote_code=True",
    tasks="hellaswag,mmlu_abstract_algebra",
    log_samples=True,
)

wandb_logger = WandbLogger(
    project="lm-eval-harness-integration", job_type="eval"
)  # or empty if wandb.init(...) already called before
wandb_logger.post_init(results)
wandb_logger.log_eval_result()
wandb_logger.log_eval_samples(results["samples"])  # if log_samples